In [1]:
# model_performance.ipynb

import cv2
import numpy as np
import os


In [4]:


from techtrack.inference.preprocessing import capture_video
from techtrack.inference.object_detection import Model
from techtrack.inference.nms import filter as nms_filter
%matplotlib inline


In [ ]:

#  compute Intersection over Union (IoU)
def compute_iou(boxA, boxB):
    """
    Computes IoU between two bounding boxes.
    boxA and boxB are in the format (x, y, w, h)
    """
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])

    interWidth = max(0, xB - xA)
    interHeight = max(0, yB - yA)
    interArea = interWidth * interHeight

    boxAArea = boxA[2] * boxA[3]
    boxBArea = boxB[2] * boxB[3]

    iou = interArea / float(boxAArea + boxBArea - interArea) if (boxAArea + boxBArea - interArea) > 0 else 0

    return iou

def read_ground_truth_bboxes(txt_file_path, image_width, image_height):
    bboxes = []
    class_ids = []
    with open(txt_file_path, 'r') as f:
        for line in f:
            # Assuming the text file format is: class_id x_center y_center width height (normalized values)
            parts = line.strip().split()
            if len(parts) >= 5:
                class_id = int(parts[0])
                x_center, y_center, w, h = map(float, parts[1:5])

                #  normalized coordinates to absolute pixel values
                x_center *= image_width
                y_center *= image_height
                w *= image_width
                h *= image_height

                #  center coordinates to top-left coordinates
                x = int(x_center - w / 2)
                y = int(y_center - h / 2)
                w = int(w)
                h = int(h)

                bboxes.append((x, y, w, h))
                class_ids.append(class_id)
    return bboxes, class_ids

#  YOLO files for Model 1
config_path_1 = '.\yolo_model_1\yolov4-tiny-logistics_size_416_1.cfg'      
weights_path_1 = '.\yolo_model_1\yolov4-tiny-logistics_size_416_1.weights' 
class_names_path_1 = '.\yolo_model_1\logistics.names' 

#  YOLO files for Model 2
config_path_2 = '.\yolo_model_2\yolov4-tiny-logistics_size_416_2.cfg'      # Update with your model's cfg file
weights_path_2 = '.\yolo_model_2\yolov4-tiny-logistics_size_416_2.weights' # Update with your model's weights file
class_names_path_2 = '.\yolo_model_2\logistics.names' # Update with your model's class names file


In [ ]:

# Initialize 
model1 = Model(config_path_1, weights_path_1, class_names_path_1)
model2 = Model(config_path_2, weights_path_2, class_names_path_2)


In [ ]:
image_dir = 'logistics'  


image_files = [f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

#  counters for performance metrics
true_positives_model1 = 0
false_positives_model1 = 0
false_negatives_model1 = 0

true_positives_model2 = 0
false_positives_model2 = 0
false_negatives_model2 = 0

iou_threshold = 0.5  # IoU threshold for detection as true positive

# Process 
for image_file in image_files:
    image_path = os.path.join(image_dir, image_file)
    txt_file_path = os.path.join(image_dir, os.path.splitext(image_file)[0] + '.txt')
    print(f"Processing image: {image_file}")
    
    # Read 
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error reading image {image_file}")
        continue
    
    image_height, image_width = image.shape[:2]
    if os.path.exists(txt_file_path):
        gt_bboxes, gt_class_ids = read_ground_truth_bboxes(txt_file_path, image_width, image_height)
    else:
        print(f"No ground truth file found for {image_file}")
        continue

        # Process with Model 1
    model1.set_frame_shape(image.shape)
    predict_output_1 = model1.predict(image)
    score_threshold = 0.5
    bboxes_1, class_ids_1, scores_1 = model1.post_process(predict_output_1, score_threshold)
    nms_iou_threshold = 0.4
    filtered_bboxes_1, filtered_class_ids_1, filtered_scores_1 = nms_filter(
        bboxes_1, class_ids_1, scores_1, nms_iou_threshold
    )
    
    # Process with Model 2
    model2.set_frame_shape(image.shape)
    predict_output_2 = model2.predict(image)
    bboxes_2, class_ids_2, scores_2 = model2.post_process(predict_output_2, score_threshold)
    filtered_bboxes_2, filtered_class_ids_2, filtered_scores_2 = nms_filter(
        bboxes_2, class_ids_2, scores_2, nms_iou_threshold
    )
    
    # Evaluate Model 1
    matched_gt_indices = []
    for i, pred_box in enumerate(filtered_bboxes_1):
        pred_class_id = filtered_class_ids_1[i]
        best_iou = 0
        best_gt_index = -1
        for j, gt_box in enumerate(gt_bboxes):
            if j in matched_gt_indices:
                continue
            if pred_class_id != gt_class_ids[j]:
                continue
            iou = compute_iou(pred_box, gt_box)
            if iou > best_iou:
                best_iou = iou
                best_gt_index = j
        if best_iou >= iou_threshold:
            true_positives_model1 += 1
            matched_gt_indices.append(best_gt_index)
        else:
            false_positives_model1 += 1
    false_negatives_model1 += len(gt_bboxes) - len(matched_gt_indices)
    
    # Evaluate Model 2
    matched_gt_indices = []
    for i, pred_box in enumerate(filtered_bboxes_2):
        pred_class_id = filtered_class_ids_2[i]
        best_iou = 0
        best_gt_index = -1
        for j, gt_box in enumerate(gt_bboxes):
            if j in matched_gt_indices:
                continue
            if pred_class_id != gt_class_ids[j]:
                continue
            iou = compute_iou(pred_box, gt_box)
            if iou > best_iou:
                best_iou = iou
                best_gt_index = j
        if best_iou >= iou_threshold:
            true_positives_model2 += 1
            matched_gt_indices.append(best_gt_index)
        else:
            false_positives_model2 += 1
    false_negatives_model2 += len(gt_bboxes) - len(matched_gt_indices)
    
    print("\n")



Processing image: -01-15-1-1-1-2-26_jpg.rf.eb62a0a5f84fcbdbc0910af072cf0072.jpg


Processing image: -01-15-1-4-1-1-14_jpg.rf.2e89a06faa1c217b95a0e0d7177e8397.jpg


Processing image: -01-15-1-4-1-4-111_jpg.rf.ead3505e4a61d1c97ae2e235b7b228d5.jpg


Processing image: -01-15-1-4-1-5-45_jpg.rf.0797388e664e12ff935b062e665f32ac.jpg


Processing image: -01-15-2-2-2-1-212_jpg.rf.c3706c341a4cbb939c5b1341793bc0d8.jpg


Processing image: -01-15-2-3-2-4-37_jpg.rf.0a2f9cb02817b70c5a56c6082535a7b6.jpg


Processing image: -01-16-1-1-1-1-20_jpg.rf.27f842b8dd4032c96ad8afc13dc855f8.jpg


Processing image: -01-16-1-2-1-3-14_jpg.rf.c2febebedae30ca6c5d67ea7adf8973b.jpg


Processing image: -01-16-2-1-2-4-20_jpg.rf.7de07f4de98267e416ac90e110467400.jpg


Processing image: -01-16-2-1-2-5-44_jpg.rf.3db2a524f3617f046a2a3b9c48a0c5ee.jpg


Processing image: -01-16-2-2-2-1-112_jpg.rf.7287af14c2a3f25b2611052832121828.jpg


Processing image: -01-16-2-5-2-4-32_jpg.rf.7b1caf338cd52204fb89c8087a84cc2f.jpg


Processing im

In [ ]:

#  Precision and Recall for Model 1
precision_model1 = true_positives_model1 / (true_positives_model1 + false_positives_model1) if (true_positives_model1 + false_positives_model1) > 0 else 0
recall_model1 = true_positives_model1 / (true_positives_model1 + false_negatives_model1) if (true_positives_model1 + false_negatives_model1) > 0 else 0

# Precision and Recall for Model 2
precision_model2 = true_positives_model2 / (true_positives_model2 + false_positives_model2) if (true_positives_model2 + false_positives_model2) > 0 else 0
recall_model2 = true_positives_model2 / (true_positives_model2 + false_negatives_model2) if (true_positives_model2 + false_negatives_model2) > 0 else 0

# Print 
print("Model 1 Performance:")
print(f"True Positives: {true_positives_model1}")
print(f"False Positives: {false_positives_model1}")
print(f"False Negatives: {false_negatives_model1}")
print(f"Precision: {precision_model1:.2f}")
print(f"Recall: {recall_model1:.2f}")

print("\nModel 2 Performance:")
print(f"True Positives: {true_positives_model2}")
print(f"False Positives: {false_positives_model2}")
print(f"False Negatives: {false_negatives_model2}")
print(f"Precision: {precision_model2:.2f}")
print(f"Recall: {recall_model2:.2f}")

# Argument for ideal model..
if precision_model1 > precision_model2 and recall_model1 > recall_model2:
    favored_model = "Model 1"
    reason = "Model 1 has higher precision and recall, indicating better overall performance in detecting objects correctly while minimizing false detections."
elif precision_model2 > precision_model1 and recall_model2 > recall_model1:
    favored_model = "Model 2"
    reason = "Model 2 has higher precision and recall, indicating better overall performance in detecting objects correctly while minimizing false detections."
elif precision_model1 > precision_model2:
    favored_model = "Model 1"
    reason = "Model 1 has higher precision, indicating it produces fewer false positives compared to Model 2."
elif precision_model2 > precision_model1:
    favored_model = "Model 2"
    reason = "Model 2 has higher precision, indicating it produces fewer false positives compared to Model 1."
elif recall_model1 > recall_model2:
    favored_model = "Model 1"
    reason = "Model 1 has higher recall, indicating it detects more true objects compared to Model 2."
else:
    favored_model = "Model 2"
    reason = "Model 2 has higher recall, indicating it detects more true objects compared to Model 1."

print(f"\nBased on the evaluation metrics, the favored model is {favored_model}.")
print(reason)


Model 1 Performance:
True Positives: 14358
False Positives: 4380
False Negatives: 22357
Precision: 0.77
Recall: 0.39

Model 2 Performance:
True Positives: 16343
False Positives: 4044
False Negatives: 20372
Precision: 0.80
Recall: 0.45

Based on the evaluation metrics, the favored model is Model 2.
Model 2 has higher precision and recall, indicating better overall performance in detecting objects correctly while minimizing false detections.
